# Part 1. Training

In [44]:
import os
import numpy as np
# import sys
# !{sys.executable} -m pip install torchvision
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib
from PIL import Image

In [45]:
#checking for device
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [46]:
print(device)

cpu


In [47]:
#Transforms
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [48]:
#Dataloader

#Path for training and testing directory
train_path='/Users/victoria/Books/covers/dataset/train'
test_path='/Users/victoria/Books/covers/dataset/test'

train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=64, shuffle=True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=32, shuffle=True
)

In [49]:
#categories
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])
classes.remove('.DS_Store')

In [50]:
print(classes)

['Crime-Thriller', 'Graphic-Novels-Anime-Manga', 'Mind-Body-Spirit', 'Romance', 'Science-Fiction-Fantasy-Horror', 'Travel-Holiday-Guides']


In [51]:
#CNN Network

class ConvNet(nn.Module):
    def __init__(self,num_classes=6):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        #Input shape= (256,3,150,150)
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (256,12,150,150)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (256,12,75,75)
        
    
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (256,20,75,75)
        
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (256,32,75,75)
        
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
        
                
        #Feed forwad function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
            
        #Above output will be in matrix form, with shape (256,32,75,75)
            
        output=output.view(-1,32*75*75)
            
            
        output=self.fc(output)
            
        return output

In [52]:
model=ConvNet(num_classes=6).to(device)

In [53]:
#Optmizer and loss function
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [54]:
num_epochs=10

In [55]:
#calculating the size of training and testing images
train_count=len(glob.glob(train_path+'/**/*.jpg'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))

In [56]:
print(train_count,test_count)

4491 1335


In [57]:
#Model training and saving best model

best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy

Epoch: 0 Train Loss: tensor(17.0483) Train Accuracy: 0.24649298597194388 Test Accuracy: 0.25617977528089886
Epoch: 1 Train Loss: tensor(3.5843) Train Accuracy: 0.451792473836562 Test Accuracy: 0.26741573033707866
Epoch: 2 Train Loss: tensor(3.2146) Train Accuracy: 0.531062124248497 Test Accuracy: 0.3303370786516854
Epoch: 3 Train Loss: tensor(2.1631) Train Accuracy: 0.6225784903139613 Test Accuracy: 0.24644194756554308
Epoch: 4 Train Loss: tensor(2.2351) Train Accuracy: 0.6470719216210198 Test Accuracy: 0.29138576779026215
Epoch: 5 Train Loss: tensor(1.1004) Train Accuracy: 0.7630817189935426 Test Accuracy: 0.32209737827715357
Epoch: 6 Train Loss: tensor(0.8311) Train Accuracy: 0.8020485415274995 Test Accuracy: 0.3048689138576779
Epoch: 7 Train Loss: tensor(0.6220) Train Accuracy: 0.8648407926965042 Test Accuracy: 0.3235955056179775
Epoch: 8 Train Loss: tensor(0.6935) Train Accuracy: 0.8470273881095525 Test Accuracy: 0.3408239700374532
Epoch: 9 Train Loss: tensor(0.5290) Train Accuracy

## Part 2. Testing a model with unknown books

In [58]:
checkpoint=torch.load('best_checkpoint.model')
model=ConvNet(num_classes=6)
model.load_state_dict(checkpoint)
model.eval()

ConvNet(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (conv3): Conv2d(20, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (fc): Linear(in_features=180000, out_features=6, bias=True)
)

In [59]:
#Transforms
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [60]:
pred_path='/Users/victoria/Books/covers/dataset/predict'

In [61]:
#prediction function
def prediction(img_path,transformer):
    
    image=Image.open(img_path)
    
    image_tensor=transformer(image).float()
    
    
    image_tensor=image_tensor.unsqueeze_(0)
    
    if torch.cuda.is_available():
        image_tensor.cuda()
        
    input=Variable(image_tensor)
    
    
    output=model(input)
    
    index=output.data.numpy().argmax()
    
    pred=classes[index]
    
    return pred

In [62]:
images_path=glob.glob(pred_path+'/*.jpg')

In [63]:
pred_dict={}

for i in images_path:
    pred_dict[i[i.rfind('/')+1:]]=prediction(i,transformer)

In [64]:
pred_dict

{'0000002.jpg': 'Mind-Body-Spirit',
 '0000167.jpg': 'Crime-Thriller',
 '0000006.jpg': 'Graphic-Novels-Anime-Manga',
 '0000076.jpg': 'Science-Fiction-Fantasy-Horror',
 '0000114.jpg': 'Romance',
 '0000058.jpg': 'Travel-Holiday-Guides',
 '0000043.jpg': 'Science-Fiction-Fantasy-Horror',
 '0000040.jpg': 'Crime-Thriller',
 '0000097.jpg': 'Travel-Holiday-Guides',
 '0000096.jpg': 'Science-Fiction-Fantasy-Horror',
 '0000055.jpg': 'Graphic-Novels-Anime-Manga',
 '0000045.jpg': 'Crime-Thriller',
 '0000131.jpg': 'Mind-Body-Spirit',
 '0000078.jpg': 'Graphic-Novels-Anime-Manga',
 '0000021.jpg': 'Graphic-Novels-Anime-Manga',
 '0000037.jpg': 'Travel-Holiday-Guides',
 '0000180.jpg': 'Science-Fiction-Fantasy-Horror',
 '0000220.jpg': 'Mind-Body-Spirit'}

## 2020 Books

In [65]:
checkpoint=torch.load('best_checkpoint.model')
model=ConvNet(num_classes=6)
model.load_state_dict(checkpoint)
model.eval()

ConvNet(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (conv3): Conv2d(20, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (fc): Linear(in_features=180000, out_features=6, bias=True)
)

In [66]:
#Transforms
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [67]:
pred_path='/Users/victoria/Books/covers/dataset/2020'

In [68]:
#prediction function
def prediction(img_path,transformer):
    image=Image.open(img_path)
    image_tensor=transformer(image).float()
    image_tensor=image_tensor.unsqueeze_(0)
    if torch.cuda.is_available():
        image_tensor.cuda()   
    input=Variable(image_tensor)
    output=model(input)
    index=output.data.numpy().argmax()
    pred=classes[index]
    return pred

In [69]:
images_path=glob.glob(pred_path+'/*.jpg')

In [70]:
pred_dict={}

for i in images_path:
    pred_dict[i[i.rfind('/')+1:]]=prediction(i,transformer)

In [71]:
pred_dict

{'002.jpg': 'Mind-Body-Spirit',
 '003.jpg': 'Crime-Thriller',
 '001.jpg': 'Science-Fiction-Fantasy-Horror',
 '004.jpg': 'Travel-Holiday-Guides'}

In [ ]:
# 0 points

## One book, different cover design 

In [72]:
pred_path='/Users/victoria/Books/covers/dataset/misery'
images_path=glob.glob(pred_path+'/*.jpg')
pred_dict={}

for i in images_path:
    pred_dict[i[i.rfind('/')+1:]]=prediction(i,transformer)
    
pred_dict

{'14.jpg': 'Mind-Body-Spirit',
 '15.jpg': 'Mind-Body-Spirit',
 '01.jpg': 'Science-Fiction-Fantasy-Horror',
 '03.jpg': 'Science-Fiction-Fantasy-Horror',
 '02.jpg': 'Science-Fiction-Fantasy-Horror',
 '12.jpg': 'Crime-Thriller',
 '06.jpg': 'Crime-Thriller',
 '07.jpg': 'Travel-Holiday-Guides',
 '13.jpg': 'Mind-Body-Spirit',
 '05.jpg': 'Travel-Holiday-Guides',
 '11.jpg': 'Travel-Holiday-Guides',
 '10.jpg': 'Graphic-Novels-Anime-Manga',
 '04.jpg': 'Science-Fiction-Fantasy-Horror',
 '09.jpg': 'Science-Fiction-Fantasy-Horror',
 '08.jpg': 'Science-Fiction-Fantasy-Horror'}